In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from nfw_project import config_paths as cfg
from pathlib import Path

In [ ]:

# Roadless areas (same as before)
roadless = gpd.read_file(cfg.USFS_GPKG, layer="roadless_area")
roadless_5070 = roadless.to_crs("EPSG:5070")
# Forest to Faucets HUC8s that intersect roadless areas
F2F = gpd.read_file(cfg.ROADLESS_ANALYSIS_GPKG, layer="F2F_RA")

In [3]:
# ensure numeric
num_cols = [
    "ACRES", "Domestic", "Industrial", "Irrigation", "Livestock", "Mining",
    "Thermo", "Public_sup", "Aquacultur", "Total_SW",
    "Ps_del_dom", "Domestic_GW", "Industri_GW", "Irrigati_GW",
    "Livestoc_GW", "Mining_GW", "Thermo_GW", "Public_sup_GW",
    "Aquacult_GW", "Total_GW"
]
for c in num_cols:
    if c in F2F.columns:
        F2F[c] = pd.to_numeric(F2F[c], errors="coerce")

# HUC8-level inventory (these are *already* intersecting RAs)
f2f_huc_summary = (
    F2F
    .groupby(["HUC_8", "HU_8_STATE"], dropna=False)
    .agg(
        HUC8_Acres=("ACRES", "first"),     # F2F polygon acres
        Total_SW=("Total_SW", "first"),
        Total_GW=("Total_GW", "first"),
        Public_sup_SW=("Public_sup", "first"),
        Public_sup_GW=("Public_sup_GW", "first"),
    )
    .reset_index()
)

# Optional: a single-line metric to say how many HUC8s intersect RAs
n_huc = f2f_huc_summary["HUC_8"].nunique()
print(f"{n_huc} unique HUC8 watersheds intersect at least one roadless area.")

614 unique HUC8 watersheds intersect at least one roadless area.


In [ ]:
# Make sure both in same CRS (use 5070 / your analysis CRS)
if roadless_5070.crs != F2F.crs:
    F2F = F2F.to_crs(("EPSG:5070")

# Optional: keep a stable ID for each roadless_5070 polygon
if "RA_ID" not in roadless_5070.columns:
    roadless_5070 = roadless_5070.reset_index(drop=True)
    roadless_5070["RA_ID"] = roadless_5070.index + 1  # simple integer ID

# Spatial overlay – splits geometries into RA–HUC pieces
ra_huc = gpd.overlay(
    roadless_5070[["RA_ID", "REGION", "FOREST", "STATE", "NAME", "geometry"]],
    F2F[["HUC_8", "HU_8_STATE", "ACRES", "Total_SW", "Total_GW",
         "Public_sup", "Public_sup_GW", "geometry"]],
    how="intersection"
)

# Area of the intersection in acres (assuming CRS is in meters)
ra_huc["INT_AREA_M2"] = ra_huc.geometry.area
ra_huc["INT_ACRES"] = ra_huc["INT_AREA_M2"] / 4046.8564224

# Fraction of the HUC8 inside the roadless_5070 area
# (Assumes F2F['ACRES'] is HUC8 polygon area in acres)
ra_huc["FRAC_HUC_IN_RA"] = ra_huc["INT_ACRES"] / ra_huc["ACRES"]


In [8]:
water_fields = ["Total_SW", "Total_GW", "Public_sup", "Public_sup_GW"]

for fld in water_fields:
    ra_huc[f"{fld}_w"] = ra_huc[fld] * ra_huc["FRAC_HUC_IN_RA"]


In [9]:
ra_water = (
    ra_huc
    .groupby(["RA_ID", "REGION", "FOREST", "STATE", "NAME"], dropna=False)
    .agg(
        RA_Acres=("INT_ACRES", "sum"),   # acres of RA that lie in F2F HUCs
        HUC8_Count=("HUC_8", "nunique"),
        Total_SW_w=("Total_SW_w", "sum"),
        Total_GW_w=("Total_GW_w", "sum"),
        Public_sup_w=("Public_sup_w", "sum"),
        Public_sup_GW_w=("Public_sup_GW_w", "sum"),
    )
    .reset_index()
)

# Example: derive a simple "total public supply" metric
ra_water["Public_sup_Total_w"] = (
    ra_water["Public_sup_w"].fillna(0) +
    ra_water["Public_sup_GW_w"].fillna(0)
)


In [14]:
top20_drinking_water = (
    ra_water
    .sort_values("Public_sup_Total_w", ascending=False)
    .head(20)
    .assign(
        Rank=lambda df: range(1, len(df) + 1)
    )[
        [
            "Rank",
            "NAME",
            "FOREST",
            "STATE",
            "REGION",
            "RA_Acres",
            "HUC8_Count",
            "Public_sup_Total_w",
        ]
    ]
)

top20_drinking_water


,Rank,NAME,FOREST,STATE,REGION,RA_Acres,HUC8_Count,Public_sup_Total_w
1511,1,Trabuco,Cleveland,CA,5,23340.780425,2,16.883782
1783,2,Sespe - Frazier,Los Padres,CA,5,41783.997815,2,14.258313
1674,3,Fish Canyon,Angeles,CA,5,29885.679322,1,11.473061
1338,4,Sheep Mountain,Angeles,CA,5,14338.414532,1,10.490040
1956,5,Pleasant View,Angeles,CA,5,26395.367604,2,7.457148
1957,6,Magic Mountain,Angeles,CA,5,15541.545578,2,5.972767
1202,7,Kings River,Sierra,CA,5,48259.704867,2,5.129107
1340,8,Strawberry Peak,Angeles,CA,5,7244.998502,2,4.660944
7390,9,Mt. Olympus,Wasatch-Cache,UT,4,7220.839087,2,4.397807
7701,10,418016,Uinta,UT,4,35240.197932,1,4.352543


In [ ]:

OUT = Path(cfg.TABLES_OUTPUT_DIR) / "roadless_F2F_tables"
OUT.mkdir(parents=True, exist_ok=True)

# HUC-level
f2f_huc_summary.to_csv(OUT / "F2F_HUC8_summary.csv", index=False)
f2f_huc_summary.to_markdown(OUT / "F2F_HUC8_summary.md", index=False)
f2f_huc_summary.to_latex(OUT / "F2F_HUC8_summary.tex", index=False)

# Roadless-level
ra_water.to_csv(OUT / "RA_F2F_water_summary.csv", index=False)
ra_water.to_markdown(OUT / "RA_F2F_water_summary.md", index=False)
ra_water.to_latex(OUT / "RA_F2F_water_summary.tex", index=False)


# F2F HUC12

In [16]:
F2F_HUC12_RA = gpd.read_file(cfg.ROADLESS_ANALYSIS_GPKG, layer="F2F2_HUC12_RA")

In [17]:
F2F_HUC12_RA.columns

Index(['Acres', 'STATES', 'HUC12', 'NAME', 'HUTYPE', 'TOHUC', 'From_', 'To_',
       'Level', 'NLCD', 'FOREST', 'PER_FOR', 'AG', 'PER_AG', 'NATCOVER',
       'PER_NATCOV', 'RIPNAT_AC', 'PER_RIPNAT', 'IMPV_AC', 'PER_IMPV', 'R_AG',
       'R_RIP', 'R_IMPV', 'R_NATCOV', 'R_Q', 'APCW', 'APCW_R', 'SW', 'SW_Pop',
       'GW', 'GW_POP', 'GL_POP', 'GL_Intakes', 'SUM_POP', 'PR', 'POP_DS',
       'IMP', 'IMP_R', 'NON_FOREST', 'PRIVATE_FOREST', 'PROTECTED_FOREST',
       'NFS_FOREST', 'FEDERAL_FOREST', 'PER_FORPRI', 'PER_FORNFS',
       'PER_FORPRO', 'PER_WFP', 'PER_IDRISK', 'PERDEV_1040_45',
       'PERDEV_1090_45', 'PERDEV_1040_85', 'PERDEV_1090_85', 'PER_Q40_45',
       'PER_Q90_45', 'PER_Q40_85', 'PER_Q90_85', 'WFP', 'IDRISK', 'DEV1040_45',
       'DEV1090_45', 'DEV1040_85', 'DEV1090_85', 'Q1040_45', 'Q1090_45',
       'Q1040_85', 'Q1090_85', 'WFP_IMP_R', 'IDRISK_R', 'Q40_45_R', 'Q40_85_R',
       'Q90_45_R', 'Q90_85_R', 'DEV40_45_R', 'DEV40_85_R', 'DEV90_45_R',
       'DEV90_85_R', 'PER_NLCD